In [ ]:
# 0. Imports and helper functions

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import statsmodels.api as sm

In [ ]:
# For time-series forecasting
# Prophet (install with: pip install prophet)
try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
except Exception:
    PROPHET_AVAILABLE = False

In [ ]:
# For machine learning models
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier, XGBRegressor

In [ ]:
# For LSTM
try:
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout
    from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
    KERAS_AVAILABLE = True
except Exception:
    KERAS_AVAILABLE = False

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 1. Load data

In [ ]:
def load_data(path='.../data/raw/superstore.csv'):
    df = pd.read_csv(path, parse_dates=['order_date','ship_date'], low_memory=False)
    # Basic type conversions
    df['postal_code'] = df['postal_code'].astype(str)
    return df

In [ ]:
# 2. Basic cleaning & EDA helpers

In [ ]:
def basic_cleaning(df):
    # Drop rows with missing order_id or sales
    df = df.dropna(subset=['order_id','sales'])
    # Convert sales, profit, discount to numeric if needed
    df['sales'] = pd.to_numeric(df['sales'], errors='coerce')
    df['profit'] = pd.to_numeric(df['profit'], errors='coerce')
    df['discount'] = pd.to_numeric(df['discount'], errors='coerce')
    df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
    # Fill simple NA
    df['segment'] = df['segment'].fillna('Unknown')
    df['region'] = df['region'].fillna('Unknown')
    return df

In [ ]:
# Quick EDA summary
def eda_summary(df):
    print('Rows:', len(df))
    print('Date range:', df['order_date'].min(), '->', df['order_date'].max())
    print(df[['sales','profit','discount','quantity']].describe())
    print('\nTop categories:')
    print(df['category'].value_counts())

In [ ]:
# 3. Forecasting sales by category / sub_category / region

In [ ]:
# Aggregate sales to monthly series per group
def aggregate_monthly(df, group_cols=['category']):
    df2 = df.copy()
    df2['year_month'] = df2['order_date'].dt.to_period('M').dt.to_timestamp()
    grouped = df2.groupby(group_cols + ['year_month']).agg({'sales':'sum'}).reset_index()
    return grouped

In [ ]:
# Prepare a dataframe for Prophet: columns ds, y
def prepare_prophet_df(grouped, group_values, group_cols=['category']):
    # group_values: dict mapping col->value, e.g. {'category':'Furniture'}
    g = grouped.copy()
    for c,v in group_values.items():
        g = g[g[c]==v]
    g = g.sort_values('year_month')
    prophet_df = g[['year_month','sales']].rename(columns={'year_month':'ds','sales':'y'})
    prophet_df['ds'] = pd.to_datetime(prophet_df['ds'])
    return prophet_df

In [ ]:
# Forecast with Prophet
def forecast_with_prophet(prophet_df, periods=12, freq='M'):
    if not PROPHET_AVAILABLE:
        raise RuntimeError('Prophet is not installed in your environment. Install prophet via pip.')
    m = Prophet()
    m.fit(prophet_df)
    future = m.make_future_dataframe(periods=periods, freq=freq)
    forecast = m.predict(future)
    return m, forecast

In [ ]:
# Simple ARIMA using statsmodels (auto_arima is not included; user can tune)
from statsmodels.tsa.arima.model import ARIMA

def forecast_with_arima(series, order=(1,1,1), steps=12):
    # series: pd.Series indexed by datetime
    model = ARIMA(series, order=order)
    res = model.fit()
    fc = res.get_forecast(steps=steps)
    pred = fc.predicted_mean
    conf = fc.conf_int()
    return res, pred, conf

In [ ]:
# LSTM pipeline (requires keras)
def build_lstm_forecast(series, n_input=12, n_epochs=50, n_batch=16, n_neurons=50):
    if not KERAS_AVAILABLE:
        raise RuntimeError('Keras/TensorFlow is not available in this environment.')
    # series: Pandas Series, index datetime, frequency monthly ideally
    data = series.values.reshape(-1,1)
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)
    generator = TimeseriesGenerator(data_scaled, data_scaled, length=n_input, batch_size=n_batch)
    model = Sequential()
    model.add(LSTM(n_neurons, activation='tanh', input_shape=(n_input,1)))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(generator, epochs=n_epochs, verbose=1)
    # Forecast next n_input steps iteratively
    pred_scaled = []
    current_batch = data_scaled[-n_input:].reshape((1,n_input,1))
    for i in range(n_input):
        pred = model.predict(current_batch)[0]
        pred_scaled.append(pred)
        current_batch = np.append(current_batch[:,1:,:], [[pred]], axis=1)
    pred = scaler.inverse_transform(np.array(pred_scaled).reshape(-1,1)).flatten()
    return model, pred

In [ ]:
# Wrapper to run forecasting for all groups and store results
def run_category_forecasts(df, group_cols=['category'], periods=12):
    grouped = aggregate_monthly(df, group_cols=group_cols)
    forecasts = []
    unique_groups = grouped[group_cols].drop_duplicates()
    for _, row in unique_groups.iterrows():
        group_values = {c: row[c] for c in group_cols}
        prophet_df = prepare_prophet_df(grouped, group_values, group_cols=group_cols)
        if len(prophet_df) < 12:
            # skip small series
            continue
        try:
            if PROPHET_AVAILABLE:
                m, forecast = forecast_with_prophet(prophet_df, periods=periods)
                fc = forecast[['ds','yhat']].tail(periods)
                fc['group'] = str(group_values)
                forecasts.append(fc)
        except Exception as e:
            print('Forecast failed for', group_values, e)
    if forecasts:
        return pd.concat(forecasts, ignore_index=True)
    else:
        return pd.DataFrame()

In [ ]:
# 4. Customer churn modelling (RFM + classification)

In [ ]:
def build_rfm(df, snapshot_date=None):
    # snapshot_date: date to compute recency relative to; default = max(order_date)+1 day
    if snapshot_date is None:
        snapshot_date = df['order_date'].max() + pd.Timedelta(days=1)
    cust = df.groupby('customer_id').agg({
        'order_date': lambda x: (snapshot_date - x.max()).days,
        'order_id': 'nunique',
        'sales': 'sum',
        'quantity': 'sum',
        'discount':'mean'
    }).rename(columns={'order_date':'recency','order_id':'frequency','sales':'monetary','discount':'avg_discount','quantity':'total_qty'}).reset_index()
    return cust

In [ ]:
# Label churn: customers with no orders in last N days
def label_churn(df, churn_days=180):
    snapshot_date = df['order_date'].max() + pd.Timedelta(days=1)
    last_purchase = df.groupby('customer_id')['order_date'].max().reset_index()
    last_purchase['days_since_last'] = (snapshot_date - last_purchase['order_date']).dt.days
    last_purchase['churn_label'] = (last_purchase['days_since_last'] > churn_days).astype(int)
    return last_purchase[['customer_id','churn_label','days_since_last']]

In [ ]:
def discount_profit_regression(df):
    # We'll run an OLS regression: profit ~ discount + category + region + quantity + sales
    # Create dummies for category and region
    d = df.copy()
    d = d.dropna(subset=['profit','discount'])
    d['log_sales'] = np.log1p(d['sales'])
    # Create dummies with limited cardinality
    cat_dummies = pd.get_dummies(d['category'], prefix='cat', drop_first=True)
    reg_dummies = pd.get_dummies(d['region'], prefix='reg', drop_first=True)
    X = pd.concat([d[['discount','quantity','log_sales']], cat_dummies, reg_dummies], axis=1)
    X = sm.add_constant(X)
    y = d['profit']
    model = sm.OLS(y, X).fit(cov_type='HC3')
    print(model.summary())
    return model

In [ ]:
# Optional: run a simple grouped analysis to find discount thresholds where profit turns negative

def discount_thresholds_by_category(df):
    d = df.copy()
    bins = [ -0.01, 0.0, 0.05, 0.1, 0.2, 0.5, 1.0]
    d['disc_bin'] = pd.cut(d['discount'], bins=bins)
    grouped = d.groupby(['category','disc_bin']).agg({'sales':'sum','profit':'sum','quantity':'sum','order_id':'nunique'}).reset_index()
    grouped['profit_margin'] = grouped['profit'] / grouped['sales']
    return grouped

In [ ]:
# 6. Putting it all together - run pipeline

In [ ]:
def run_full_pipeline(path='superstore.csv'):
    df = load_data(path)
    df = basic_cleaning(df)
    eda_summary(df)

    print('\nRunning category forecasts (Prophet if available) - this may take time...')
    cat_fc = run_category_forecasts(df, group_cols=['category'], periods=12)
    print('Forecasts generated for categories:', cat_fc['group'].nunique() if not cat_fc.empty else 0)

    print('\nTraining churn model...')
    churn_model = train_churn_model(df)

    print('\nRunning discount->profit regression...')
    reg_model = discount_profit_regression(df)

    print('\nComputing discount thresholds per category...')
    disc_summary = discount_thresholds_by_category(df)

    return {
        'data': df,
        'category_forecasts': cat_fc,
        'churn_model': churn_model,
        'profit_regression': reg_model,
        'discount_summary': disc_summary
    }